# Онтологический анализатор нормативных документов
## Генерация и оценка онтологий с использованием LLM

Импорт основных компонентов из модуля `src.llm_ontology`:

In [ ]:
# Установка зависимостей
!pip install owlready2 openai scikit-learn matplotlib networkx pandas

In [ ]:
import json
import pandas as pd
from IPython.display import display, Markdown
import matplotlib.pyplot as plt
import warnings

# Импорт основных классов из модуля
from src.llm_ontology import OntologyGenerator, OntologyEvaluator

# Настройка отображения
%matplotlib inline
warnings.filterwarnings("ignore", category=UserWarning)
plt.style.use('ggplot')

## 1. Подготовка тестовых данных

In [ ]:
law_text = """
Закон о персональных данных

Статья 1. Основные понятия
1. Персональные данные - любая информация, относящаяся к прямо или косвенно определенному физическому лицу.
2. Оператор - государственный орган, юридическое лицо, обрабатывающее персональные данные.

Статья 2. Принципы обработки
1. Обработка должна осуществляться на законной основе.
2. Объем данных должен соответствовать заявленным целям.

Статья 3. Права субъекта
1. Субъект имеет право на доступ к своим данным.
2. Субъект может требовать удаления данных.
"""

with open("test_law.txt", "w", encoding="utf-8") as f:
    f.write(law_text)

# Gold standard для оценки
gold_standard = {
    "elements": [
        "ПерсональныеДанные", 
        "Оператор", 
        "СубъектДанных",
        "обрабатывает",
        "имеет"
    ]
}

with open("gold_standard.json", "w", encoding="utf-8") as f:
    json.dump(gold_standard, f, ensure_ascii=False, indent=2)

## 2. Генерация онтологии

In [ ]:
# Инициализация генератора
generator = OntologyGenerator(
    llm_model="gpt-3.5-turbo",
    api_key="ваш_api_ключ",  # Замените на реальный ключ
    base_url=None  # Для локальных моделей укажите URL
)

# Генерация онтологии
display(Markdown("### Генерация онтологии из текста закона..."))
ontology = generator.generate_from_text(law_text, "privacy_law")

if ontology:
    ontology.save(file="privacy_law.owl", format="rdfxml")
    display(Markdown(f"✅ Успешно сгенерировано {len(list(ontology.classes()))} классов и {len(list(ontology.object_properties()))} свойств"))
else:
    display(Markdown("❌ Ошибка генерации онтологии"))

## 3. Визуализация онтологии

In [ ]:
if ontology:
    evaluator = OntologyEvaluator("privacy_law.owl")
    display(Markdown("### Визуализация иерархии классов"))
    evaluator.visualize_hierarchy()
else:
    display(Markdown("❌ Невозможно визуализировать: онтология не сгенерирована"))

## 4. Оценка качества онтологии

In [ ]:
if ontology:
    # Генерация вопросов компетентности
    display(Markdown("### Генерация вопросов компетентности (CQ)..."))
    cq_list = generator.generate_competency_questions(ontology, num_questions=5)
    
    if cq_list:
        display(Markdown("#### Сгенерированные CQ:"))
        for i, cq in enumerate(cq_list, 1):
            display(Markdown(f"{i}. {cq}"))
    else:
        display(Markdown("❌ Не удалось сгенерировать CQ"))
    
    # Расчет метрик
    display(Markdown("### Расчет метрик качества"))
    
    with open("gold_standard.json", "r", encoding="utf-8") as f:
        gold_standard = json.load(f)
    
    # Подготовка данных системы
    system_output = {
        "elements": [
            c.name for c in ontology.classes()
        ] + [
            p.name for p in ontology.object_properties()
        ]
    }
    
    # Вычисление метрик
    cq_coverage = evaluator.calculate_cq_coverage(cq_list, cq_list[:4]) if cq_list else 0.0
    error_index = evaluator.detect_ontology_errors()
    f1 = evaluator.evaluate_f1(gold_standard, system_output)
    depth = evaluator.calculate_hierarchy_depth()
    
    # Семантическая оценка
    g_eval_prompt = """Оцените от 1 до 5, насколько класс '{class_name}' соответствует документу:
    Определение класса: {class_definition}
    Контекст документа: {document_text}"""
    semantic_score = evaluator.semantic_evaluation(law_text, g_eval_prompt)
    
    # Вывод результатов
    metrics_data = {
        "Метрика": ["Покрытие CQ", "Индекс ошибок", "F1-Score", "Глубина иерархии", "Семантическая оценка"],
        "Значение": [cq_coverage, error_index, f1, depth, semantic_score],
        "Целевое": [0.85, 0.05, 0.8, "4-6", 4.0]
    }
    
    display(Markdown("#### Таблица метрик:"))
    display(pd.DataFrame(metrics_data).style\
        .bar(subset=['Значение'], align='mid', color=['#d65f5f', '#5fba7d']))
else:
    display(Markdown("❌ Невозможно оценить: онтология не сгенерирована"))

## 5. Генерация отчета

In [ ]:
if ontology:
    report = evaluator.generate_report()
    
    display(Markdown("### Полный отчет о качестве онтологии"))
    display(Markdown(f"```json\n{json.dumps(report, indent=2, ensure_ascii=False)}\n```"))
    
    # Сохранение отчета
    with open("ontology_report.json", "w", encoding="utf-8") as f:
        json.dump(report, f, ensure_ascii=False, indent=2)
    
    display(Markdown("### Отчет сохранен в `ontology_report.json`"))
else:
    display(Markdown("❌ Невозможно сгенерировать отчет"))